# Explore ANS using toy examples

Given the Decimal System:
- Alphabet: $X = \{0, \dots, 9\}$, uniformly distributed

The Entropy per symbol: $$H_p(X_i) = E_p[-\log_2 P(X_i)] = E_p[-\log_2 P(\dfrac{1}{10})] = \log_2 10 = 3.32 \text{ bits}$$

The expected code word length using optimal symbol encoding (average depth of the tree for alphabet X): $$E_p[l(X_i)] = 3.4 \text{ bits}$$

Can we achieve a better compression than the optimal symbol code?


In [5]:
from typing import Iterator

def encode_better_than_symbol_coding(msg:list[int], base:int)->int:
    #create integer to represent the compressed message
    compressed = 1
    for symb in msg:
        assert symb < base, f"Symbol {symb} is greater than base {base}"
        compressed  = compressed * base + symb # multiply by base and add the new symbol
    return compressed

def decode_better_than_symbol_coding(compressed:int, base:int)->Iterator[int]:
    while compressed != 1: # while the compressed number is not 1 (starting number)
        yield compressed % base # get the last digit
        compressed = compressed // base # remove the last digit


initial_msg, base = [2,3,4,5,6,7,8,9,2,3,4,5,6,7,8,9,2,3,4,5], 10
e = encode_better_than_symbol_coding(initial_msg, base)
d = decode_better_than_symbol_coding(e, base)

print(f"initial_msg: {initial_msg}")
print(f"encoded: {e} | binary representation: {e:b}")
print(f"decoded: {list(d)[::-1]}")


print(f"bitrate: {e.bit_length() / len(initial_msg) :.2f}")

initial_msg: [2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5]
encoded: 123456789234567892345 | binary representation: 1101011000101001110100111111011010011101100101100000100110101111001
decoded: [2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5]
bitrate: 3.35


**Observations of the above compression algo**: Conversion between different numeral systems
1. operates as a stack (LIFO)
2. amortizes compressed bits over symbols (could be better than symbol codes)
3. optimally compresses a sequence of symbols if these symbols:
    - are from the same alphabet
    - uniformly distributed over the alphabet thus statistically independent iid (no correlations between symbols) 


### Improving our coding algorithm

In [6]:
from typing import Iterator

class Coder:
    def __init__(self):
        self.compressed = 1

    def encode(self, symbol:int, base:int)->int:
        #create integer to represent the compressed message
        assert symbol < base, f"Symbol {symbol} is greater than base {base}"
        self.compressed = self.compressed * base + symbol

    def decode(self, base:int)->Iterator[int]:
        symbol = self.compressed % base # get the last digit
        self.compressed //= base # remove the last digit
        return symbol
    


# initial message
import random
initial_msg = [random.randint(0, 9) for _ in range(5)]
coder = Coder()
print(f"Initial message: {initial_msg}")

# Encode the message with different bases
print("\nEncoding process:")
# Base must be at least symbol + 1
bases = [symbol + 1 for symbol in initial_msg]

for symbol, base in zip(initial_msg, bases):
    coder.encode(symbol, base)
    print(f"Symbol: {symbol}, Base: {base} -> Compressed: {coder.compressed}")

print(f"\nFinal encoded value: {coder.compressed}")
print(f"Binary representation: {coder.compressed:b}")
e = coder.compressed.bit_length()

# Decode the message
print("\nDecoding process:")
decoded = []

for base in reversed(bases):
    symbol = coder.decode(base)
    decoded.append(symbol)
    print(f"Base: {base} -> Symbol: {symbol}")

print(f"\nDecoded message: {decoded[::-1]}")
print(f"Bitrate: {e / len(initial_msg) :.2f} bits per symbol")

Initial message: [6, 1, 4, 5, 0]

Encoding process:
Symbol: 6, Base: 7 -> Compressed: 13
Symbol: 1, Base: 2 -> Compressed: 27
Symbol: 4, Base: 5 -> Compressed: 139
Symbol: 5, Base: 6 -> Compressed: 839
Symbol: 0, Base: 1 -> Compressed: 839

Final encoded value: 839
Binary representation: 1101000111

Decoding process:
Base: 1 -> Symbol: 0
Base: 6 -> Symbol: 5
Base: 5 -> Symbol: 4
Base: 2 -> Symbol: 1
Base: 7 -> Symbol: 6

Decoded message: [6, 1, 4, 5, 0]
Bitrate: 2.00 bits per symbol


Observations:
- you DO NOT need to encode every symbol from the same alphabet (you can reduce/change the base to compress better)
    - this drastically reduces/improves the bitrate
- the symbols dont have to be uniformly distributed